In [ ]:
from ultralytics import YOLOv10
model=YOLOv10('/DeepStream-Yolo/EldetectionClassification.pt')
model.export('engine',half=True,opset=17)



In [ ]:
e={'0':'black_core','1':'corner','2':'crack','3':'finger','4':'fragment','5':'horizental_dislocation','6':'printing_error','7':'scratch','8':'short_circuit','9':'star_crack','10':'thick_line','11':'vertical_dislocation','12':'defective','13':'good'}

In [ ]:
import cv2

cur_frame = 0
prev_frame = 0

cap = cv2.VideoCapture(0)

# Confidence threshold
confidence = 0.7

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Calculate and display FPS
    cur_frame = cv2.getTickCount()
    fps = cv2.getTickFrequency() / (cur_frame - prev_frame)
    prev_frame = cur_frame
    cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    results = model(frame)  

    for result in results:
        for box in result.boxes:
            if box.conf >= confidence:
                x1, y1, x2, y2 = box.xyxy[0]
                # Draw rectangle
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                # Draw text
                cv2.putText(frame, str(int(box.cls)), (int(x1), int(y2) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Camera', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
